## Chapter 5: Quantum Fourier Transform

### Introduction to Forest SDK

To run the Forest SDK locally, you should have the Quantum Virtual Machine(QVM) and the Quil Compiler running in server mode on your computer. It is better to run them from the terminal. We have to make sure that the ports we are using are available. For a start, we need to import some libraries. We shall also import qutip and latex tools to facilitate visualization of circuits.

In [104]:
# Import Libraries

from pyquil import Program, get_qc
from pyquil.gates import *
import cmath
import matplotlib.pyplot as plt
import numpy as np
from pyquil.api import ForestConnection
from pyquil.api import WavefunctionSimulator
from pyquil.api import QVMConnection
from qutip import Bloch
from tempfile import mkdtemp

qvm = QVMConnection()

In [105]:
# Verify QVM and Quilc are available

from pyquil import get_qc, Program
from pyquil.gates import CNOT, Z
from pyquil.api import local_forest_runtime

prog = Program(Z(0), CNOT(0, 1))

with local_forest_runtime():
    qvm = get_qc('9q-square-qvm')
    results = qvm.run_and_measure(prog, trials=10)


In [130]:
# Test
qvm = QVMConnection()

hello_qubit = Program()

print(qvm.wavefunction(hello_qubit))

(1+0j)|0>


In [128]:
# construct a Bell State program
p = Program(H(0), CNOT(0, 1))

In [131]:
# run the program on a QVM
qc = get_qc('9q-square-qvm')
result = qc.run_and_measure(p, trials=10)
print(result[0])
print(result[1])

[1 0 1 1 0 1 0 1 1 0]
[1 0 1 1 0 1 0 1 1 0]


In [116]:
# Function to generate circuits in pyQuil. However, this requires a full instaltion of Latex. 
# It is possible using texlive on Ubuntu. This can be done with tlmgr install quantikz. 
#In general, managing LaTeX distributions is tricky, and depends on your OS.

def plot_circuit(circuit):
    latex_diagram = to_latex(circuit)
    tmp_folder = mkdtemp()
    with open(tmp_folder + '/circuit.tex', 'w') as f:
        f.write(latex_diagram)
    proc = subprocess.Popen(['pdflatex', '-shell-escape', tmp_folder + '/circuit.tex'], cwd=tmp_folder)
    proc.communicate()
    image = plt.imread(tmp_folder + '/circuit.png')
    shutil.rmtree(tmp_folder)
    plt.axis('off')
    return plt.imshow(image)

In [123]:
import numpy as np
from pyquil import Program, get_qc
from pyquil.gates import *

π = np.pi


## Quantum Fourier Transform

In [119]:
# Quantum Fourier Transform 

state_prep = Program(X(0))

qft = Program()
qft += H(0)
qft += CPHASE(π/2, 1, 0)
qft += H(1)
qft += CPHASE(π/4, 2, 0)
qft += CPHASE(π/2, 2, 1)
qft += H(2);

In [120]:
# ADd dummy qubits

from pyquil.api import WavefunctionSimulator

add_dummy_qubits = Program(I(1), I(2))  # The identity gate I has no affect

wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(state_prep + add_dummy_qubits)
print(wavefunction)

(1+0j)|001>


In [125]:
# 3 qubit QFT Program: DFT of [0, 1, 0, 0, 0, 0, 0, 0], using pyQuil:

from math import pi

def qft1(q0, q1, q2):
    p = Program()
    p += [SWAP(q0, q2),
          H(q0),
          CPHASE(-pi / 2.0, q0, q1),
          H(q1),
          CPHASE(-pi / 4.0, q0, q2),
          CPHASE(-pi / 2.0, q1, q2),
          H(q2)]
    return p

print(qft1(0, 1, 2))

SWAP 0 2
H 0
CPHASE(-pi/2) 0 1
H 1
CPHASE(-pi/4) 0 2
CPHASE(-pi/2) 1 2
H 2



In [126]:
compute_qft_prog = state_prep + qft1(0, 1, 2)
wavefunction = wf_sim.wavefunction(compute_qft_prog)
print(wavefunction.amplitudes)

[ 3.53553391e-01+0.j          2.50000000e-01-0.25j
  2.16489014e-17-0.35355339j -2.50000000e-01-0.25j
 -3.53553391e-01+0.j         -2.50000000e-01+0.25j
 -2.16489014e-17+0.35355339j  2.50000000e-01+0.25j      ]


In [127]:
# Verify qft by inverse FFT

from numpy.fft import ifft
ifft(wavefunction.amplitudes, norm="ortho")


array([ 0.00000000e+00+0.00000000e+00j,  1.00000000e+00+5.45603965e-17j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00-1.53080850e-17j,
        0.00000000e+00+0.00000000e+00j, -7.85046229e-17-2.39442265e-17j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00-1.53080850e-17j])

After ignoring the terms that are on the order of 1e-17, we get [0, 1, 0, 0, 0, 0, 0, 0], which was our input. 